In [209]:
import numpy as np
import random

In [210]:
import networkx as nx
G = nx.read_edgelist("ENZYMES_g102.edges", nodetype=int)

In [251]:
#MAX_FITNESS = G.order() # Tal y como está definido, el máximo valor que puede tomar el fitness es el número de nodos del grafo
MAX_FITNESS = 1
MAX_ITER = 10
SAMPLE_SIZE = 1
INITIAL_SAMPLE_SIZE = 300

In [212]:
# Diccionario para no recalcular fitness:
dic = {}

In [213]:
def subsets(a):
  l = len(a)
  n = l - 1
  x = [0] * n
  f = [j for j in range(l)]
  r = list()
  
  while n:
    m = [0] + x # insert 0 at the beginning of the bit string forms
    subset = [a[i] for i in range(l) if m[i] == 1] # generate the subset
    
    # and use the subset to generate the subsets for when 1 is inserted at the beginning
    k = [a[0]] + subset
    
    r.append(subset) # add to the table all subsets of bits strigs with prefix 0
    r.append(k) # add to the table all subsets of bits strings with prefix 1
    
    # initialization
    # this is where we choose which j we want to change
    j = f[0]
    f[0] = 0
    
    # if j == n we terminate because we have all the possible bit strings
    if j == n:
      break
     
    # update our array when j != n
    f[j] = f[j + 1]
    f[j + 1] = j + 1
    
    x[j] = 1 - x[j] # complement coordinate j and return to line 8 if j != n
  
  return r

In [214]:
def is_cover(sol, G):
    for u, v in G.edges():
        if sol[u - 1] == 0 and sol[v - 1] == 0:
            return False
    return True

In [241]:
def fitness(sol):
    print("oye por favor")
    n = G.order()
    if is_cover(sol, G):
        return n - sum(sol)
    else:
        if sum(sol) >= (2 * n // 3):
            print("estoy en sitio bueno")
            zeros_index = []
            for i in range(len(sol)):
                if sol[i] == 0:
                    zeros_index.append(i)
                    #sol[i] = 1
                    #if is_cover(sol, G):
                    #    print(sol)
                    #    break
                    #sol[i] = 0
            r = subsets(zeros_index)
            r.sort(key=len)
            # r[0] es el conjunto vacío
            for subset in r[1:]:
                for index in subset:
                    sol[index] = 1 # Esto modifica sol porque se pasa por referencia
                if is_cover(sol, G):
                    print(f"found fitness {n - sum(sol)}")
                    return n - sum(sol)
                for index in subset:
                    sol[index] = 0
            print("calavera 1")
            return 0
        else:
            print("calavera 2")
            return 0

In [247]:
(G.order() - 10) / G.order()

0.7619047619047619

In [253]:
def fitness2(sol):
    if is_cover(sol, G):
        print("found")
        n = G.order()
        return (n - sum(sol)) / n
    else:
        return 0

In [255]:
def fitness3(sol):
    print("oye por favor")
    n = G.order()
    if is_cover(sol, G):
        return (n - sum(sol)) / sum(sol)
    else:
        if sum(sol) >= (2 * n // 3):
            print("estoy en sitio bueno")
            zeros_index = []
            for i in range(len(sol)):
                if sol[i] == 0:
                    zeros_index.append(i)
                    #sol[i] = 1
                    #if is_cover(sol, G):
                    #    print(sol)
                    #    break
                    #sol[i] = 0
            r = subsets(zeros_index)
            r.sort(key=len)
            # r[0] es el conjunto vacío
            for subset in r[1:]:
                for index in subset:
                    sol[index] = 1 # Esto modifica sol porque se pasa por referencia
                if is_cover(sol, G):
                    print(f"found fitness {n - sum(sol)}")
                    return (n - sum(sol)) / sum(sol)
                for index in subset:
                    sol[index] = 0
            print("calavera 1")
            return 0
        else:
            print("calavera 2")
            return 0

In [260]:
def f(sol):
    fit = dic.get(tuple(sol))
    if fit == None:
        fit = fitness3(sol)
        dic[tuple(sol)] = fit
    return fit

In [217]:
def hamming_distance(sol1, sol2):
    n = len(sol1)
    #print(f"sol1 = {sol1}\nsol2 = {sol2}")
    assert(n == len(sol2))
    return [sol1[i] == sol2[i] for i in range(n)].count(False)

In [218]:
def inverse_distance(sol1, sol2, dist):
    d = dist(sol1, sol2)
    return 1 / d if d != 0 else 2 # Realmente la forma de manejar el infinito depende de la distancia
# El 2 funciona para Hammings porque no puede haber distancias d 0 < d < 1 (es decir, tales que 1 / d > 2).

In [219]:
inverse_distance([1,0,1,0], [0,1,0,1], hamming_distance)

0.25

In [220]:
def w(sol1, sol2, A, dist):
    assert(len(sol1) == len(sol2))
    sum = 0
    for sol in A:
        assert(len(sol) == len(sol1))
        sum += inverse_distance(sol1, sol, dist)
    assert(sum > 0) # Creo que se puede demostrar que esto no va a ser 0.
    inv = inverse_distance(sol1, sol2, dist)
    res = inv / sum
    #print(f"inv: {inv}\nsum: {sum}\nres: {res}")
    return res

In [229]:
def g(sol, A, dist):
    sum = 0
    for point in A:
        wei = w(sol, point, A, dist)
        fit = f(point)
        #print(f"\nwei: {wei} \n fit: {fit}")
        sum += wei * fit
    print(f"\n\n g: {sum}\n\n")
    return sum

In [258]:
def sample(k, A, dist, skip = []):
    n = G.order()
    sample = []
    while len(sample) < k:
        x = [random.choice([0, 1]) for _ in range(n)]
        if x in sample or x in skip:
            continue
        #y = random.randint(0, MAX_FITNESS)
        y = random.uniform(0, MAX_FITNESS)
        g_val = g(x, A, dist)
        if g_val > y:
            #print(fitness(x))
            print(f"Muestra añadida {x}\ncon valor de g: {g_val} y valor de y: {y}")
            sample.append(x)
    return sample

In [223]:
def init(k):
    n = G.order()
    init = []
    while len(init) < k:
        sol = [random.choice([0, 1]) for _ in range(n)]
        if sol not in init: # Esto se evita usando un set. Aún no tengo claro si necesito índices, o si el set es eficiente en python.
            init.append(sol)
    return init

def max_k_fitness(set_sols, k):
    sor = sorted([(fitness(elem), elem) for elem in set_sols])
    return [elem for (_,elem) in sor][-k:]

def max_fitness(set_sols):
    max_value = fitness(set_sols[0])
    max_elem = set_sols[0]
    for elem in set_sols[1:]:
        f = fitness(elem)
        if f > max_value:
            max_value = f
            max_elem = elem
    return max_elem, max_value

def next_gen(curr_gen, k):
    new = sample(k, skip = curr_gen)
    return max_k_fitness(curr_gen + new, k)

def algorithm():
    gen = init(SAMPLE_SIZE)
    for _ in range(MAX_ITER):
        gen = next_gen(gen, SAMPLE_SIZE)
    return max_fitness(gen)

In [224]:
def max_f(A):
    assert(len(A) > 0)
    max_value = f(A[0])
    max_elem = A[0]
    for elem in A[1:]:
        fit = f(elem)
        if fit > max_value:
            max_value = fit
            max_elem = elem
    return max_elem, max_value

In [225]:
def alg(sample_size, num_iter, initial_sample_size, dist):
    dic.clear()
    A = init(initial_sample_size)
    for i in range(num_iter):
        print(f"iter {i}")
        A.extend(sample(sample_size, A, dist, skip = A))
    return max_f(A)

In [261]:
elem, fit = alg(sample_size = SAMPLE_SIZE, num_iter = MAX_ITER, initial_sample_size = INITIAL_SAMPLE_SIZE, dist = hamming_distance)
print(f'Máximo valor de fitness ({fit}) encontrado para {elem}, con {sum(elem)} nodos.')

print(is_cover(elem, G))


### A partir de aquí son formas de rellenar


num_ceros = G.order() - sum(elem)
print(num_ceros, "ceros")

iter 0
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
calavera 2
oye por favor
cala